# Cartopy Workshop: Geospatial Mapping in Python

This practical workshop introduces Cartopy, a Python library for geospatial visualization and mapping, along with related tools for handling geospatial data. We will explore core mapping workflows including projections, topographic data, and shapefile visualization.

> **Learning objectives:**
>
- Generate maps with custom projections and geographic features
- Integrate topographic raster and vector datasets
- Visualize shapefiles and geodatabases

---
## Requirements
required packages:
```
cartopy geopandas matplotlib pygmt rasterio
```
Further documentation: [Cartopy](https://scitools.org.uk/cartopy/docs/latest/), [GeoPandas](https://geopandas.org/), [PyGMT](https://www.pygmt.org/), [Rasterio](https://rasterio.readthedocs.io/en/stable/)

In [ ]:
#This is only required if we're running in Google Colab
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!conda update conda -y -q
!conda config --prepend channels conda-forge
!conda install -q -y --prefix /usr/local python=3.8 pygmt

import sys
import os
sys.path.append('/usr/local/lib/python3.13/site-packages')
os.environ["GMT_LIBRARY_PATH"]="/usr/local/lib"

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pygmt    # for accessing global topo datasets
import rasterio # for reading GeoTIFF
import geopandas as gpd

from pyproj import Geod

## 1. Map of Australia: Plotting Features and Locations

Cartopy allows us to draw regional maps with built-in coastlines, state boundaries, and other features. Below, we plot Australia, add state boundaries, river networks, and highlight Brisbane, Queensland.

In [ ]:
# Coordinates for Brisbane
brisbane_lon, brisbane_lat = 153.0251, -27.4698

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([110, 155, -45, -10]) # Australia extent

# Add physical features
ax.add_feature(cfeature.LAND, facecolor='whitesmoke')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.BORDERS, linestyle=':', lw=1)
ax.add_feature(cfeature.STATES, edgecolor='k', ls='--', lw=0.7)
ax.add_feature(cfeature.RIVERS, alpha=0.4)

# Add gridlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Mark Brisbane
ax.plot(brisbane_lon, brisbane_lat, 'ro', markersize=8, label='Brisbane')
ax.text(brisbane_lon + 1, brisbane_lat, 'Brisbane', fontsize=12, color='red')

plt.title('Australia with State Boundaries and Brisbane')
plt.show()

## 2. Alternative Projections

Cartopy supports a range of map projections for scientific and publication-quality figures. For a full list, see the [Cartopy projection documentation](https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html).

> **Tip:**
>
- When using a projection, always specify the original data CRS (usually `ccrs.PlateCarree`) with `transform` when plotting points or setting extents.

Below, we use a Robinson projection.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.Robinson())
ax.set_extent([110, 155, -45, -10], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, facecolor='w')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, edgecolor='k', ls='--')
ax.add_feature(cfeature.RIVERS, alpha=0.4)
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

ax.plot(brisbane_lon, brisbane_lat, marker='o', color='red', markersize=8,
        transform=ccrs.PlateCarree(), label='Brisbane')
ax.text(brisbane_lon + 1, brisbane_lat, 'Brisbane', transform=ccrs.PlateCarree(), color='red')

plt.title('Australia: Robinson Projection')
plt.show()

## 3. Zooming in: Mapping the Brisbane Region

The coordinates for the Brisbane region are `[152.8, 153.3, -27.7, -27.2]`. Adjust the map extent for regional focus.

In [ ]:
### Type your code here

##### 3.1 Adding a scalebar to a map
There is no built-in method to add a scalebar, so we have to manually calculate it

In [ ]:
def add_scalebar(ax, length_km, location=(0.1, 0.05), linewidth=3, text_fontsize=10, color='k'):
    """
    Adds a scale bar to the map.
    ax : The axis to draw a scalebar on.
    length_km : The length of the scalebar in kilometers.
    location : Tuple (x, y) in axes fraction coordinates (0 = left/bottom, 1 = right/top).
    linewidth : Thickness of the scale bar.
    """
    # Get axis extent in data coordinates
    x0, x1, y0, y1 = ax.get_extent(crs=ccrs.PlateCarree())
    # Start point in data coordinates
    x_start = x0 + (x1 - x0) * location[0]
    y_start = y0 + (y1 - y0) * location[1]
    # Determine longitude offset for desired distance
    geod = Geod(ellps="WGS84")
    lon2, lat2, _ = geod.fwd(x_start, y_start, 90, length_km * 1000)  # 90 deg = due east
    # Plot scale bar
    ax.plot([x_start, lon2], [y_start, y_start], color=color, linewidth=linewidth, transform=ccrs.PlateCarree())
    # Text label
    ax.text((x_start + lon2)/2, y_start - 0.01 * (y1 - y0), f'{length_km} km',
            ha='center', va='top', fontsize=text_fontsize, color=color,
            transform=ccrs.PlateCarree())

In [ ]:
# Coordinates for Brisbane
brisbane_lon, brisbane_lat = 153.0251, -27.4698

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([110, 155, -45, -10]) # Australia extent

# Add physical features
ax.add_feature(cfeature.LAND, facecolor='whitesmoke')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.BORDERS, linestyle=':', lw=1)
ax.add_feature(cfeature.STATES, edgecolor='k', ls='--', lw=0.7)
ax.add_feature(cfeature.RIVERS, alpha=0.4)

# Add gridlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Mark Brisbane
ax.plot(brisbane_lon, brisbane_lat, 'ro', markersize=8, label='Brisbane')
ax.text(brisbane_lon + 1, brisbane_lat, 'Brisbane', fontsize=12, color='red')

add_scalebar(ax, length_km=500, location=(0.1, 0.05), linewidth=3, text_fontsize=10, color='black')
# add_scalebar(ax, length_km=100, location=(0.137, 0.05), linewidth=3, text_fontsize=0, color='white')

plt.title('Australia with State Boundaries and Brisbane')
plt.show()



## 4. Plotting Topographic Data: DEMs with PyGMT and Rasterio

You can visualize digital elevation models (DEMs) using Cartopy, PyGMT (for data access), and Rasterio (for local GeoTIFFs).

### 4.1: Download and Plot Topography with PyGMT

PyGMT provides access to the GEBCO global topographic dataset. Here, we download a high-resolution DEM for the Brisbane area and plot it as a raster overlay.

In [ ]:
### get the DEM data from pygmt
region = [152.8, 153.3, -27.7, -27.2]
grid = pygmt.datasets.load_earth_relief(resolution="15s", region=region)

In [ ]:
#### plot the figure
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent(region)


# Plot DEM as a raster
dem = ax.imshow(grid.data, extent=[grid.lon.min(), grid.lon.max(), grid.lat.min(), grid.lat.max()],
                origin='lower', cmap='terrain', transform=ccrs.PlateCarree(), vmin=-10, vmax=300, zorder=0)

### add the coastlines on top
ax.add_feature(cfeature.STATES, linestyle=':', lw=1, zorder=1)

### add a contour at 0m (sea level)
contour = ax.contour(grid.lon, grid.lat, grid.data, levels=[0], colors='black', linewidths=1, transform=ccrs.PlateCarree(), zorder=2)



# Add gridlines and location
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

ax.plot(brisbane_lon, brisbane_lat, 'ro', markersize=8)
ax.text(brisbane_lon, brisbane_lat + 0.01, 'Brisbane', fontsize=12, color='red')

# Colorbar for elevation
cbar = plt.colorbar(dem, ax=ax, orientation='vertical', pad=0.05, aspect=30)
cbar.set_label('Elevation (m)', fontsize=12)
plt.title('Brisbane Region Topography (GEBCO DEM)')
plt.show()

### 4.2: Plotting Local GeoTIFF DEMs with Rasterio

You can plot local topographic rasters (GeoTIFF format) with Rasterio. Download DEMs from sources like [OpenTopography](https://portal.opentopography.org/raster?opentopoID=OTSRTM.122019.4326.1).

In [ ]:
Aus_DEM_geotiff = 'https://github.com/TarynScharf/PythonWorkshop/raw/refs/heads/main/SEG/Data_for_exercises/output_SRTM15Plus.tif'

with rasterio.open(Aus_DEM_geotiff) as src:
    dem_data = src.read(1)
    dem_bounds = src.bounds
    dem_extent = [dem_bounds.left, dem_bounds.right, dem_bounds.bottom, dem_bounds.top]
    dem_crs = src.crs

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(region)

ax.imshow(dem_data, extent=dem_extent, origin='upper', cmap='terrain', transform=ccrs.PlateCarree(), vmin=-10, vmax=300, zorder=0)

### add the coastlines on top
ax.add_feature(cfeature.STATES, linestyle=':', lw=1, zorder=1)


# Add gridlines and location
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

### Add our location on top
ax.plot(brisbane_lon, brisbane_lat, 'ro', markersize=8)
ax.text(brisbane_lon, brisbane_lat + 0.01, 'Brisbane', fontsize=12, color='red')
plt.title('DEM from Local GeoTIFF (SRTM15+)')
plt.show()

### Try changing the location that is plotted to Airlie Beach
- The extent is `[148.65, 148.83 -20.32, -20.176]`
- long and lat is `[148.714, -20.268]`

In [ ]:
### Type your code here

## 5. Plotting Shapefiles and Geodatabases

Shapefiles and geodatabases are standard formats for vector geospatial data. We use GeoPandas to read and plot these datasets.

### 5.1: Shapefile Polygons (Australian LGAs)

Australian Local Government Area (LGA) shapefiles are available from the [Australian Bureau of Statistics](https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files).

In [ ]:
Australia_LGAs = gpd.read_file('https://github.com/TarynScharf/PythonWorkshop/raw/refs/heads/main/SEG/Data_for_exercises/LGA_2025_AUST_GDA2020/LGA_2025_AUST_GDA2020.shp')

In [ ]:
## Check the crs
print("DEM CRS:", dem_crs)
print("Australia LGAs CRS:", Australia_LGAs.crs)

#### Convert the crs to EPSG:4326
Different CRS can be found [here](https://epsg.io/)

In [ ]:
Australia_LGAs.to_crs(epsg=4326, inplace=True)
print("Australia LGAs CRS updated:", Australia_LGAs.crs)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([152.8, 153.3, -27.7, -27.2])


ax.add_feature(cfeature.LAND, facecolor='w')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, edgecolor='k', ls='--')
ax.add_feature(cfeature.RIVERS, alpha=0.4)

gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

### Add our location
ax.plot(brisbane_lon, brisbane_lat, 'ro', markersize=8)
ax.text(brisbane_lon, brisbane_lat + 0.01, 'Brisbane', fontsize=12, color='red')

# Plot all LGAs as outlines
Australia_LGAs.plot(ax=ax, edgecolor='blue', facecolor='none', linewidth=0.5, transform=ccrs.PlateCarree())

# Highlight Brisbane LGA
Brisbane_LGA = Australia_LGAs[Australia_LGAs['LGA_NAME25'] == 'Brisbane']

Brisbane_LGA.plot(
    ax=ax, edgecolor='red', facecolor='lime', linewidth=1.5, transform=ccrs.PlateCarree())

plt.title('Brisbane Region: LGA Boundaries')
plt.show()

### Now try plotting another LGA
- The list of LGAs can be obtained by  'Australia_LGAs['LGA_NAME25']'

In [ ]:
### Type your code here

### 5.2: Plotting other geospatial data (e.g., Mineral Deposits)

Point and line data can be visualized similarly. For example, mineral deposit locations can be colored by commodity type. Data is available from [Data.gov.au](https://www.data.gov.au/data/dataset/14e96462-b029-469a-9af8-06410f39589b).

In [ ]:
aus_mineral_deposits = gpd.read_file('https://github.com/TarynScharf/PythonWorkshop/tree/main/SEG/Data_for_exercises/Mineral_Deposits_v01_20130729/Mineral_Deposits.gdb')
print("Australia Mineral Deposits CRS:", aus_mineral_deposits.crs)

In [ ]:
aus_mineral_deposits.to_crs(epsg=4326, inplace=True)
print("Australia Mineral Deposits CRS updated:", aus_mineral_deposits.crs)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([110, 155, -45, -10])

### add our usual features
ax.add_feature(cfeature.LAND, facecolor='w')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, edgecolor='k', ls='--')
ax.add_feature(cfeature.RIVERS, alpha=0.4)

gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Plot mineral deposits, color-coded by commodity
aus_mineral_deposits.plot(column='CT_COMMODNAME', ax=ax, markersize=20, alpha=0.7, transform=ccrs.PlateCarree(),
                         legend=True, legend_kwds={'loc': 'lower left', 'bbox_to_anchor': (-0.1, -1.2), 'ncol': 3})

### plot our location
ax.plot(brisbane_lon, brisbane_lat, 'rx', markersize=8)
ax.text(brisbane_lon + 1, brisbane_lat, 'Brisbane', fontsize=12, color='red')

plt.title('Australia: Mineral Deposit Locations')
plt.show()

#### Now try plotting by a different column
e.g. `OPERATING_STATUS`
- The column names can be obtained by: `aus_mineral_deposits.columns`
- The  first couple of rows can be obtained by: `aus_mineral_deposits.head()`

In [ ]:
### Type your code here